# Chapter7. 합성곱 신경망(CNN)

## 7.1 전체 구조

* CNN
    * 합성곱 계층(convolutional layer)
    * 풀링 계층(pooling layer)
    * 완전 연결 계층(fully-connected layer; Affine layer)

<img src="./images/7-1.png" width="400" height="400"/>
[7-1]

<img src="./images/7-2.png" width="400" height="400"/>
[7-2]

## 7.2 합성곱 계층

* 패딩(padding)
* 스트라이드(stride)

### 7.2.1 완전연결 계층의 문제점

* 문제점 : 데이터의 형상이 무시된다
    * 이미지의 경우 세로,가로,채널(색상)으로 구성된 3차원 데이터이지만 완전연결 계층에서는 1차원으로 바꿔야한다

> 합성곱 계층의 장점 : 형상을 유지한다

* 특징 맵(feature map) : 합성곱 계층의 입출력 데이터
    * 입력 데이터 : 입력 특징 맵
    * 출력 데이터 : 출력 특징 맵

### 7.2.2 합성곱 연산

* 합성곱 연산 = 필터 연산
* 필터는 커널이라고도 한다

<img src="./images/7-3.png" width="400" height="400"/>
[7-1]

* 단일 곱센-누산 진행

<img src="./images/7-4.png" width="400" height="400"/>
[7-4]

* 완전연결 신경망에서 가중치와 편향은 존재
* CNN에서는 필터의 매개변수가 가중치에 해당
* 필터는? => 필터는 항상 1 x 1 형태이다

<img src="./images/7-5.png" width="500" height="500"/>
[7-5]

### 7.2.3 패딩

* 패딩 : 합성곱 연산 수행 전 입력 데이터 주변을 특정 값으로 채우는 것
* 패딩은 주로 출력 크기를 조정할 목적으로 사용한다

<img src="./images/7-6.png" width="400" height="400"/>
[7-6]

### 7.2.4 스트라이드

* 스트라이드(stride) : 필터를 적용하는 위치 간격

<img src="./images/7-7.png" width="400" height="400"/>
[7-7]

#### (패딩과 스트라이드에 따른 출력크기 수치화)

입력 크기가 $(H, W)$, 필터 크기가 $(FH, FW)$ 출력 크기가 $(OH, OW)$ 패딩이 $P$ 스트라이드가 $S$ 일 때

<img src="./images/7.1.png" width="200" height="200"/>
[7.1]

* +1 왼쪽 분수가 정수가 되게 해야한다

### 7.2.5 3차원 데이터의 합성곱 연산

<img src="./images/7-8.png" width="400" height="400"/>
[7-8]

(3차원)

<img src="./images/7-9.png" width="400" height="400"/>
[7-9]

* 3차원 합성곱 연산에서 입력 데이터의 채널 수와 필터의 채널 수가 같아야 함
* 필터 자체의 크기는 자유롭지만, 모든 필터가 같은 크기여야함

### 7.2.6 블록으로 생각하기

<img src="./images/7-10.png" width="400" height="400"/>
[7-10]

* 위 예에서 출력 데이터는 한 장의 특징 맵
* 다수의 채널을 내보내려면? => 필터(가중치)를 다수 사용하기
* 즉, 필터의 수도 고려해야 함

<img src="./images/7-11.png" width="400" height="400"/>
[7-11]

* 필터의 가중치 데이터는 4차원 : (출력 채널 수, 입력 채널 수, 높이, 너비)

#### (+편향)

<img src="./images/7-12.png" width="400" height="400"/>
[7-12]

### 7.2.7 배치 처리